## Function Calling

https://github.com/azure-ai-foundry/foundry-samples/blob/main/samples/agent-catalog/msft-agent-samples/foundry-agent-service-sdk/meeting-prep-agent/python/template.py

In [ ]:
import json
import datetime
import socket
from typing import Any, Callable, Set, Dict, List, Optional
from dotenv import load_dotenv  
import json
import datetime
from typing import Any, Callable, Set, Dict, List, Optional
import os
# Load the .env file  
load_dotenv(override=True) 


# These are the user-defined functions that can be called by the agent.


def fetch_current_datetime(format: Optional[str] = None) -> str:
    """
    Get the current time as a JSON string, optionally formatted.

    :param format (Optional[str]): The format in which to return the current time. Defaults to None, which uses a standard format.
    :return: The current time in JSON format.
    :rtype: str
    """
    current_time = datetime.datetime.now()

    # Use the provided format if available, else use a default format
    if format:
        time_format = format
    else:
        time_format = "%Y-%m-%d %H:%M:%S"

    time_json = json.dumps({"current_time": current_time.strftime(time_format)})
    print(f"Current time: {current_time.strftime(time_format)}")
    return time_json


def fetch_weather(location: str) -> str:
    """
    Fetches the weather information for the specified location.

    :param location (str): The location to fetch weather for.
    :return: Weather information as a JSON string.
    :rtype: str
    """
    # In a real-world scenario, you'd integrate with a weather API.
    # Here, we'll mock the response.
    mock_weather_data = {"New York": "Sunny, 25°C", "London": "Cloudy, 18°C", "Tokyo": "Rainy, 22°C"}
    weather = mock_weather_data.get(location, "Weather data not available for this location.")
    weather_json = json.dumps({"weather": weather})
    return weather_json



# # Define user functions
user_functions = {fetch_weather, fetch_current_datetime}

agent_name = os.environ["AZURE_AI_AGENT_NAME_04"]

In [ ]:
import os, time
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import FunctionTool
from azure.ai.agents.models import FunctionTool, ToolSet, ListSortOrder

# Retrieve the project endpoint from environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]

# Initialize the AIProjectClient
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

# Initialize the FunctionTool with user-defined functions

functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)
project_client.agents.enable_auto_function_calls(toolset)





In [ ]:

agent_name = os.environ["AZURE_AI_AGENT_NAME_04"]
agent_list = project_client.agents.list_agents()

agent = None
if agent_list:
    for agent_object in agent_list:
        if agent_object.name == agent_name:
            agent = agent_object

if not agent:
    print(f"Agent '{agent_name}' not found in the project.")
    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name=agent_name,
        instructions="You are a helpful agent with tools to get the current time, weather, and PC name.  Based on the user request, you can call the functions to get the current time, weather, and PC name, or use your general knowledge to answer a question without calling a tool",
        tools=functions.definitions,
    )
    print(f"Created agent, ID: {agent.id}")
else:
    print(f"Agent '{agent_name}' already exists, ID: {agent.id}")

In [ ]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

# Send a message to the thread
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="Hello, what is datetime and weather information in New York?",
)
print(f"Created message, ID: {message['id']}")

In [ ]:
# Create and process a run for the agent to handle the message
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, ID: {run.id}")

# Poll the run status until it is completed or requires action
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action":
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        tool_outputs = []
        for tool_call in tool_calls:
            if tool_call.name == "fetch_weather":
                output = fetch_weather("New York")
                tool_outputs.append({"tool_call_id": tool_call.id, "output": output})
        project_client.agents.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

print(f"Run completed with status: {run.status}")

# Fetch and log all messages from the thread
messages = project_client.agents.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Role: {message['role']}, Content: {message['content']}")

In [ ]:
# Send a message to the thread
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="great, tell me a joke",
)
print(f"Created message, ID: {message['id']}")
# Create and process a run for the agent to handle the message
run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, ID: {run.id}")

# Poll the run status until it is completed or requires action
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action":
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        tool_outputs = []
        for tool_call in tool_calls:
            if tool_call.name == "fetch_weather":
                output = fetch_weather("New York")
                tool_outputs.append({"tool_call_id": tool_call.id, "output": output})
        project_client.agents.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

print(f"Run completed with status: {run.status}")

# Fetch and log all messages from the thread
messages = project_client.agents.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Role: {message['role']}, Content: {message['content']}")

In [36]:
from azure.ai.evaluation import CoherenceEvaluator, AzureOpenAIModelConfiguration

load_dotenv(override=True) 

model_config = AzureOpenAIModelConfiguration(
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
        api_key=os.environ.get("AZURE_OPENAI_KEY"),
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
        api_version=os.environ.get("AZURE_API_VERSION"),
    )


In [ ]:
# Extract user and agent messages for coherence evaluation
import json
from IPython.display import display, Markdown

coherence = CoherenceEvaluator(model_config=model_config, threshold=3)
user_message = None
agent_message = None
messages = project_client.agents.messages.list(thread_id=thread.id)
for message in messages:
    print(message.role)
    if str(message.role) == "MessageRole.USER" and user_message is None:
        user_message = message.content
    elif str(message.role) in ("MessageRole.AGENT") and agent_message is None:
        agent_message = message.content

if user_message and agent_message:
    print("checking coherence")
    coherence_score = coherence(query=user_message, response=agent_message)
    #print(f"Coherence score: {coherence_score}")
    json_string = json.dumps(coherence_score, default=str, indent=4)
    display(Markdown(f"```json\n{json_string}\n```"))
else:
    print("Could not find both user and agent messages for coherence evaluation.")






## Intent Resolution

In [ ]:
from azure.ai.evaluation import IntentResolutionEvaluator

intent_resolution = IntentResolutionEvaluator(model_config=model_config, threshold=3)
intent_resolution(
    query=user_message,
    response=agent_message
)